In [17]:
import os
import json
import requests
from openai import OpenAI
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from dotenv import load_dotenv

load_dotenv()

# Initialize the Azure OpenAI client
client = OpenAI(
    base_url=os.getenv("AZURE_OPENAI_WEBSEARCH_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

# Define the deployment you want to use for your chat completions API calls

deployment_name = "gpt-4o-mini"

# Simplified weather data
# WEATHER_DATA = {
#     "tokyo": {"temperature": "10", "unit": "celsius"},
#     "san francisco": {"temperature": "72", "unit": "fahrenheit"},
#     "paris": {"temperature": "22", "unit": "celsius"},
#     "seoul": {"temperature": "15", "unit": "celsius"}
# }

WEATHER_DATA = {
    "tokyo": None,
    "san francisco": None,
    "paris": None,
    "seoul": None
}

# Simplified timezone data
TIMEZONE_DATA = {
    "tokyo": "Asia/Tokyo",
    "san francisco": "America/Los_Angeles",
    "paris": "Europe/Paris",
    "seoul" : "Asia/Seoul"
}

def get_current_weather(location, unit=None):
    """Get the current weather for a given location"""
    location_lower = location.lower()

    print(f"get_current_weather called with location: {location}, unit: {unit}")  
    
    for key in WEATHER_DATA:
        if key in location_lower:
            print(f"Weather data found for {key}")  
            # weather = WEATHER_DATA[key]

            geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={key}&count=1&language=ko"
            geo = requests.get(geo_url).json()

            lat, lon = geo['results'][0]['latitude'], geo['results'][0]['longitude']

            weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m"
            data = requests.get(weather_url).json()

            return json.dumps({
                "location": location,
                "temperature": data["current"]["temperature_2m"],
                "unit": None
            })
    
    print(f"No weather data found for {location_lower}")  
    return json.dumps({"location": location, "temperature": "unknown"})

# def get_current_weather(location, unit=None):
#     """Get the current weather for a given location"""
#     location_lower = location.lower()
#     print(f"get_current_weather called with location: {location}, unit: {unit}")  
    
#     for key in WEATHER_DATA:
#         if key in location_lower:
#             print(f"Weather data found for {key}")  
#             weather = WEATHER_DATA[key]
#             return json.dumps({
#                 "location": location,
#                 "temperature": weather["temperature"],
#                 "unit": unit if unit else weather["unit"]
#             })
    
#     print(f"No weather data found for {location_lower}")  
#     return json.dumps({"location": location, "temperature": "unknown"})

def get_current_time(location):
    """Get the current time for a given location"""
    print(f"get_current_time called with location: {location}")  
    location_lower = location.lower()
    
    for key, timezone in TIMEZONE_DATA.items():
        if key in location_lower:
            print(f"Timezone found for {key}")  
            current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
            return json.dumps({
                "location": location,
                "current_time": current_time
            })
    
    print(f"No timezone data found for {location_lower}")  
    return json.dumps({"location": location, "current_time": "unknown"})

def run_conversation():
    # Initial user message
    messages = [{"role": "user", "content": "seoul의 현재 시간과 날씨, tokyo의 현재 시간과 날씨, paris의 현재 시간과 날씨를 리스트 형색으로 나열해서 알려줘"}]

    # Define the functions for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco MUST be translated into English",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco MUST be translated into English",
                        },
                    },
                    "required": ["location"],
                },
            }
        }
    ]

    # First API call: Ask the model to use the functions
    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Model's response:")  
    print(response_message)  

    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function call: {function_name}")  
            print(f"Function arguments: {function_args}")  
            
            if function_name == "get_current_weather":
                function_response = get_current_weather(
                    location=function_args.get("location"),
                    unit=function_args.get("unit")
                )
            elif function_name == "get_current_time":
                function_response = get_current_time(
                    location=function_args.get("location")
                )
            else:
                function_response = json.dumps({"error": "Unknown function"})
            
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
    else:
        print("No tool calls were made by the model.")  

    # Second API call: Get the final response from the model
    final_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
    )

    return final_response.choices[0].message.content

# Run the conversation and print the result
print(run_conversation())

Model's response:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_GApPw5vwGqVOrScnY24yk8fJ', function=Function(arguments='{"location": "Seoul"}', name='get_current_time'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_iFFBTQ8t3VOYjgNwOlXOvWeb', function=Function(arguments='{"location": "Seoul"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_eSpStMBqjXMoUXQWzufvLuIW', function=Function(arguments='{"location": "Tokyo"}', name='get_current_time'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_kk5c3T0OVFCrNUhfOxWRoLrV', function=Function(arguments='{"location": "Tokyo"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_rqwIigTv0jbMplET7Sgccd5B', function=Function(arguments='{"location": "Paris"}', name='get_current_time'), type='funct